# Directly accessing SDO data using the DRMS


Install `drms` Python module from [PyPI](https://pypi.python.org/pypi/drms):

    pip install drms

Resources:

- HMI Science Nugget: http://hmi.stanford.edu/hminuggets/?p=1757
- Documentation: http://drms.readthedocs.io/


## Create DRMS client instance

In [ ]:
import drms
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
client = drms.Client('kis')
client

## List available series

In [ ]:
client.series('hmi\.')

## Accessing metadata #1

Querying all metadata for HMI magnetograms from 2011 to 2015 using one magnetogram per day: 

In [ ]:
keys = client.query('hmi.m_720s[2011.01.01-2015.01.01@1d]', key=drms.const.all)

In [ ]:
keys

In [ ]:
keys.columns

In [ ]:
si = client.info('hmi.m_720s')
si.keywords.loc['DSUN_OBS']

In [ ]:
#print(keys['T_REC'])
t_rec = drms.to_datetime(keys['T_REC'])
#print(t_rec)

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)

ax[0].plot(t_rec, keys['DSUN_OBS'])
ax[0].set_ylabel('Distance [m]')

ax[1].plot(t_rec, keys['CRLT_OBS'])
ax[1].set_ylabel('$B_0$ angle [deg]')

ax[1].set_xlabel('Date')
plt.setp(ax[1].get_xticklabels(), rotation=30)
plt.tight_layout()
plt.show()

## Accessing metadata #2

Querying all metadata for 5 days of HMI Dopplergrams, separated by 15 minutes:

In [ ]:
keys = client.query('hmi.v_45s[2013.02.01/5d@15m]', key=drms.const.all)

In [ ]:
keys.columns

In [ ]:
t_rec = drms.to_datetime(keys['T_REC'])

fig, ax = plt.subplots(3, 1, sharex=True)

ax[0].plot(t_rec, keys['DSUN_OBS'])
ax[0].set_ylabel('Distance [m]')

ax[1].plot(t_rec, keys['OBS_VR'])
ax[1].set_ylabel('$v_r$ [m/s]')

ax[2].plot(t_rec, keys['OBS_VN'])
ax[2].set_ylabel('$v_n$ [m/s]')

ax[2].set_xlabel('Date')
plt.setp(ax[2].get_xticklabels(), rotation=30)
plt.tight_layout()
plt.show()

# Accessing data files

The following only works if you have direct access to the KIS NFS, e.g., when running Python on one of our servers.

Querying metadata and FITS file locations for HMI Dopplergrams, magnetograms, continuum intensity maps:

In [ ]:
keys_dopp, segs_dopp = client.query('hmi.v_45s[2013.01.01_TAI/1d@1h]', key=drms.const.all, seg=drms.const.all)
keys_cont, segs_cont = client.query('hmi.ic_720s[2013.01.01_TAI/1d@1h]', key=drms.const.all, seg=drms.const.all)
keys_mag, segs_mag = client.query('hmi.m_720s[2013.01.01_TAI/1d@1h]', key=drms.const.all, seg=drms.const.all)

Using the record time as table indices:

In [ ]:
keys_dopp.index = segs_dopp.index = drms.to_datetime(keys_dopp['T_REC'])
keys_cont.index = segs_cont.index = drms.to_datetime(keys_cont['T_REC'])
keys_mag.index = segs_mag.index = drms.to_datetime(keys_mag['T_REC'])
segs_dopp

### Reading FITS files from NFS:

In [ ]:
fname_cont = segs_cont['continuum']['2013-01-01 15:00:00']
fname_mag = segs_mag['magnetogram']['2013-01-01 15:00:00']
print('Filenames:', fname_cont, fname_mag, sep='\n')

print('Reading continuum intensity map...')
cont = fits.getdata(fname_cont)
print('Continuum:', cont.shape, cont.dtype)

print('Reading magnetogram...')
mag = fits.getdata(fname_mag)
print('Magnetogram:', mag.shape, mag.dtype)

### Some simple plots

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(6, 9))

im_cont = ax[0].imshow(cont, cmap='gray')
fig.colorbar(im_cont, ax=ax[0])

im_mag = ax[1].imshow(mag, cmap='gray', vmin=-500, vmax=500)
fig.colorbar(im_mag, ax=ax[1])

fig.tight_layout()
fig.show()

In [ ]:
ax[0].set_xlim(1600, 2800)
ax[1].set_ylim(700, 1900)